In [1]:
import pandas as pd
import numpy as np
# import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# from xgboost import XGBClassifier
# from catboost import CatBoostClassifier
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import recall_score, confusion_matrix, precision_score, f1_score, accuracy_score, classification_report

In [3]:
df = pd.read_csv('../data/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [4]:
df.drop(labels=df[df['tenure'] == 0].index, axis=0, inplace=True)
df[df['tenure'] == 0].index

Index([], dtype='int64')

In [5]:
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [6]:
def object_to_int(dataframe_series):
    if dataframe_series.dtype=='object':
        dataframe_series = LabelEncoder().fit_transform(dataframe_series)
    return dataframe_series

In [7]:
df = df.apply(lambda x: object_to_int(x))
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,5365,0,0,1,0,1,0,1,0,0,...,0,0,0,0,0,1,2,29.85,2504,0
1,3953,1,0,0,0,34,1,0,0,2,...,2,0,0,0,1,0,3,56.95,1465,0
2,2558,1,0,0,0,2,1,0,0,2,...,0,0,0,0,0,1,3,53.85,156,1
3,5524,1,0,0,0,45,0,1,0,2,...,2,2,0,0,1,0,0,42.30,1399,0
4,6500,0,0,0,0,2,1,0,1,0,...,0,0,0,0,0,1,2,70.70,924,1


In [8]:
X = df.drop(columns = ['Churn'])
y = df['Churn'].values

In [9]:
import numpy as np
from xgboost import XGBClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.metrics import make_scorer, f1_score

# Định nghĩa hàm đánh giá F1 score
f1_scorer = make_scorer(f1_score, average='macro')

# Định nghĩa không gian tìm kiếm các tham số
param_space = {
    'learning_rate': Real(0.01, 1.0, 'log-uniform'),
    'n_estimators': Integer(50, 200),
    'max_depth': Integer(1, 20),
    'subsample': Real(0.5, 1.0, 'uniform'),
    'colsample_bytree': Real(0.5, 1.0, 'uniform'),
}

# Thiết lập mô hình XGBoost
xgb = XGBClassifier(random_state=42)

# Thiết lập BayesSearchCV
opt_xgb = BayesSearchCV(
    estimator=xgb,
    search_spaces=param_space,
    n_iter=50,  # Số lượng lần lặp của Bayesian optimization
    scoring=f1_scorer,
    cv=5,  # Sử dụng 5-fold cross-validation
    n_jobs=-1,  # Sử dụng tất cả các CPU cores có sẵn
    random_state=42
)

# Thực hiện tìm kiếm tham số tối ưu
opt_xgb.fit(X, y)

# In ra tham số tốt nhất và F1 score tương ứng
print("Tham số tối ưu của XGBoost:", opt_xgb.best_params_)
print("F1 score tốt nhất của XGBoost:", opt_xgb.best_score_)


Tham số tối ưu của XGBoost: OrderedDict([('colsample_bytree', 1.0), ('learning_rate', 0.09152408714955712), ('max_depth', 1), ('n_estimators', 182), ('subsample', 0.5692858501056642)])
F1 score tốt nhất của XGBoost: 0.7330554647808929


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.20, random_state = 40, stratify=y)

In [11]:
best_params = opt_xgb.best_params_

In [12]:
best_params

OrderedDict([('colsample_bytree', 1.0),
             ('learning_rate', 0.09152408714955712),
             ('max_depth', 1),
             ('n_estimators', 182),
             ('subsample', 0.5692858501056642)])

In [13]:
# Import các thư viện cần thiết
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score, classification_report, confusion_matrix

# Giả sử best_params chứa các tham số tối ưu tìm được trước đó
best_params = {
    'learning_rate': 0.1,
    'n_estimators': 100,
    'max_depth': 3,
    'subsample': 1.0,
    'max_features': None
}

# Tạo mô hình Gradient Boosting
clf_gb = GradientBoostingClassifier(**best_params)

# Huấn luyện mô hình
clf_gb.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred_gb = clf_gb.predict(X_test)

# Đánh giá mô hình bằng F1-score
f1_gb = f1_score(y_test, y_pred_gb, average='weighted')  # Dùng average='weighted' cho bài toán đa lớp
print(f'Gradient Boosting F1 Score: {f1_gb:.2f}')

# Báo cáo chi tiết
print('Gradient Boosting Classification Report:')
print(classification_report(y_test, y_pred_gb))

print('Gradient Boosting Confusion Matrix:')
print(confusion_matrix(y_test, y_pred_gb))

# Nếu bạn muốn lưu mô hình để sử dụng sau này, bạn có thể sử dụng thư viện joblib
import joblib
joblib.dump(clf_gb, 'gradient_boosting_model.pkl')


Gradient Boosting F1 Score: 0.80
Gradient Boosting Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.91      0.88      1033
           1       0.69      0.53      0.60       374

    accuracy                           0.81      1407
   macro avg       0.77      0.72      0.74      1407
weighted avg       0.80      0.81      0.80      1407

Gradient Boosting Confusion Matrix:
[[945  88]
 [175 199]]


['gradient_boosting_model.pkl']